In [ ]:
# Import Dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
from datetime import datetime
import json

In [ ]:
# Set up for the scrape ##### Change to match yours
executable_path = {'executable_path': 'C:/Users/Joe/bin/chromedriver.exe'}

In [ ]:
StartTime= datetime.now()

#Starting the file with a left bracket
fileName='DiscData.json'
with open(fileName,'a') as outfile:
    outfile.write("[")

Main_url = 'https://www.dgcoursereview.com/browse.php?ext_results=&cname=&designer=&country=1&city=&state=&zip_distance=&zipcode=&holes=0&length_min=5&length_max=20000&num_reviews=3&rating_min=&rating_max=&yem=&yex=&cndtn=&mtees=&mpins=&holetype=0&teetype=0&cf=&terrain%5B%5D=1&terrain%5B%5D=2&terrain%5B%5D=3&coursetype%5B%5D=1&landscape%5B%5D=1&landscape%5B%5D=2&landscape%5B%5D=3&private=1&paytoplay=1'

for page in range(1,210):
    Scrape_Dict=[]
    
    #Calling the page with a list of 20 courses
    url=Main_url+"&page="+str(page)
    browser = Browser('chrome', **executable_path, headless=True)
    browser.visit(url)
    time.sleep(2)
    html = browser.html
    browser.quit()

    #Parsing the html with beautiful soup
    soup = bs(html, 'html.parser')
    Table=soup.find('table', class_='form_cell')
    Rows=Table.find_all('tr', class_= 'note')
    PageArray=[]

    #Grabbing data for the 20 courses displayed
    for i in range(20):
        row=Rows[i]
        PageArray.append([])
        PageArray[i].append(row.find_all('td')[0].find('a')['href'].strip())#Course Link
        PageArray[i].append(row.find_all('td')[0].find('a').text.strip())#Course Name
        CityState=row.find_all('td')[1].text.strip().split(',')
        PageArray[i].append(CityState[0].strip())#Course City
        PageArray[i].append(CityState[1].strip())#Course State
        PageArray[i].append(int(row.find_all('td')[2].text.strip()))#Holes
        PageArray[i].append(row.find_all('td')[4].text.strip())#Tee Type
        PageArray[i].append(row.find_all('td')[5].find('img')['title'])# Landscape
        PageArray[i].append(row.find_all('td')[6].find('img')['title']) #Terrain
        if row.find_all('td')[0].find('img', src="images/icon_water.gif") != None:
            PageArray[i].append(True)#Has Water
            PageArray[i].append(int(row.find_all('td')[0].find('img', src="images/icon_water.gif")['title'].split()[4]))
        else:
            PageArray[i].append(False)#No Water
            PageArray[i].append(0)

        if row.find_all('td')[0].find('img', src="images/icon_private.gif") != None:
            PageArray[i].append(True)#Private
        else:
            PageArray[i].append(False)

        if row.find_all('td')[0].find('img', src="images/icon_paytoplay_gold.gif") != None:
            PageArray[i].append(True)#Pay
        else:
            PageArray[i].append(False)

    #Opening each course page to grab the rest of the data
    for x in PageArray:
        browser = Browser('chrome', **executable_path, headless=True)
        browser.visit(x[0])
        time.sleep(2)
        html = browser.html
        browser.quit()
        soup = bs(html, 'html.parser')

        #This had to be dynamic for finding course length becuase of inconsistencies in page build per course
        Length=0
        for j in range (3,7):
            if soup.find('div', id='content_right').find_all('div', id='content_tab')[2].find_all('div', id='content_infobar')[j].find('span',class_='category').text=="Course Length: ":

                Length=int(soup.find('div', id='content_right').find_all('div', id='content_tab')[2].find_all('div', id='content_infobar')[j].find('span', id='crse_length').text.split()[0])

                break

        #Formatting the data as json
        Course={
            "name": x[1],
            "location": {
                "city": x[2],
                "state": x[3]
            },
            "private": x[10],
            "pay": x[11],
            "holes": x[4],
            "water": x[8],
            "water_holes": x[9],
            "tee_type": x[5],
            "terrain": x[7],
            "landscape": x[6],
            "rating": float(soup.find('span', class_='score_num').text),
            "reviews": int(soup.find('div', id='content_right').find_all('div', id='content_tab')[0].find_all('div', id='content_infobar')[0]               .find('a').text),
            "played": int(soup.find('div', id='content_right').find_all('div', id='content_tab')[0].find_all('div', id='content_infobar')[1]                .find('a').text),
            "favorited": int(soup.find('div', id='content_right').find_all('div', id='content_tab')[0].find_all('div', id='content_infobar')                [2].find('a').text),
            "wishlisted": int(soup.find('div', id='content_right').find_all('div', id='content_tab')[0].find('div', id='content_infobar_nb')                .find('a').text),
            "year_established": soup.find('div', id='content_right').find_all('div', id='content_tab')[2].find_all('div',                                   id='content_infobar')[1].find('span', class_='right').text.strip(),
            "length_ft": Length
        }
        Scrape_Dict.append(Course)

    #Writing the course data into the json file
    for course in Scrape_Dict:
        with open(fileName,'a') as outfile:
            json.dump(course, outfile)
            outfile.write(",")

#Closing the json file with a bracket
with open(fileName,'a') as outfile:
    outfile.write("]")
print(f'The scrape took {datetime.now()-StartTime}')